# Political Labelling

This script determines the political affiliation (left, center, right) of each user in our sample by analyzing the retweets they have made.

We use a list of political influencers previously categorized as left, center, or right by La Silla Vacia, a Colombian news outlet. For each user, we tally the number of retweets they've made (excluding retweets with comments) that correspond to each influencer. From this data, we calculate the total number of tweets associated with each political category.

This process is carried out on tweets from the "Paro Nacional" period and on tweets that are not from this period, across three sections:

1. Paro Nacional tweets
2. Tweets not related to the Paro Nacional
3. Outputs

In [1]:
import pickle
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import scipy.sparse as sp
import os

path = r"/mnt/disk2/Data"

# Cargar Datos de congresistas y partidos

In [36]:
partidos = pd.read_excel(os.path.join(path,"clasificacion_partidos_v1.xlsx"),"Sheet1")
partidos = partidos.loc[:,['codigo_partido','ideologia']]

congresistas = pd.read_excel(os.path.join(path,"Twitter Congresistas.xlsx"),"Sheet1")
congresistas = congresistas.loc[:,('Partido', 'Twitter')]

# We load the tweets_lite DataFrame for the analysis
retweets = pd.read_pickle(os.path.join(path,"Tweets_DataFrames","retweets.gzip"), compression='gzip')

# We load the map that relates an ID to a political Label
with open(os.path.join(path,"Pickle","User_Dicts","mapa.pkl"), "rb") as file:
    mapa = pickle.load(file)

In [37]:
def partidos_dict(x):
    
    # Manualmente, reemplazar el nombre del partido por el ID
    d = {
        "PartidoSocialdeUnidadNacional":20050002,#Partido de la U
        "CentroDemocrático":20130001,#Centro Democrático
        "ListadelaDecencia":20180008,#Lista de la Decencia
        "AlianzaVerde":20090002,#Alianza Verde
        "PartidoConservadorColombiano":18490002,#Partido Conservador Colombiano
        "PartidoLiberalColombiano":18480001,#Partido Liberal Colombiano
        "ColombiaJustaLibres":20170001,#Colombia Justa Libres
        "PoloDemocráticoAlternativo":20050001,#Polo Democrático Alternativo
        "PartidoCambioRadical":20030001,#Partido Cambio Radical
        "ConsejoComunitariodeComunidadesNegrasPlayaRenaciente":20180014,#Consejo Comunitario Playa Renaciente
        "Comunes":20170003,#Partido Comunes
        "MovimientoAlternativoIndígenaySocial":20130002,#MAIS
        "ConsejoComunitarioLaMamuncia":20180029,#ConsejoComunitariolaMamuncia(wrongnameinexternaldatabase)
        "CoaliciónAlternativaSantandereana":20180042,#CoaliciónAlternativaSantandereana
        "MovimientoIndependientedeRenovaciónAbsoluta":20000036,#MIRA
        "OpciónCiudadana":20090001, #Partido Opción Ciudadana
    }
    
    return d.get(x,0)

def idelogias_dict(x):
    
    # Agrega la ideologia
    d = {
        1:'Izquierda',
        2:'Derecha',
        3:'Centro',
        4:None
    }
    
    return d.get(x,None)

In [40]:
# Diccionario con ID Nombre de twittero
user_name = (
    retweets[['Referenced Tweet Author ID', 'Referenced Tweet Author Name']]
    .drop_duplicates()
    .astype({'Referenced Tweet Author ID':'float64'})
    .set_index('Referenced Tweet Author ID')
    .to_dict()['Referenced Tweet Author Name']
)

# Limpiar nombres para agregar el ID de partido
congresistas.Partido = congresistas.Partido.str.replace(' ','')
congresistas['Partido ID'] = congresistas['Partido'].apply(partidos_dict)
congresistas_new = congresistas.merge(partidos,left_on='Partido ID', right_on='codigo_partido', how = 'left')

# Obetenemos la ideología
congresistas_new['Afiliacion'] = congresistas_new.ideologia.apply(idelogias_dict)

# Partidos que no aparecen en la base del CEDE
print("Partidos que no aparecen en la base del CEDE o no tienen ideologia")
for p in congresistas_new[congresistas_new['Afiliacion'].isna()].Partido.unique():
    print(p)

Partidos que no aparecen en la base del CEDE o no tienen ideologia
ColombiaJustaLibres
AutoridadesIndígenasdeColombia
FuerzaAlternativaRevolucionariadelComún
OpciónCiudadana
CoaliciónAlternativaSantandereana
ConsejoComunitariodeComunidadesNegrasPlayaRenaciente
ConsejoComunitarioLaMamuncia


In [41]:
# Dado que estos partidos no fueron, se agrega la ideologia a mano
# partido FARC y Autoridades Índigena vistos como izquierda

congresistas_new.loc[congresistas_new['Afiliacion']=='nada','Afiliacion'] = 'Izquierda'

# Pasamos datos importantes a un diccionario
mapa_2 = (
    congresistas_new.merge(retweets, left_on='Twitter', right_on='Referenced Tweet Author Name')
    .loc[:,('Referenced Tweet Author ID','Afiliacion')]
    .drop_duplicates()
    .astype({'Referenced Tweet Author ID':'float64'})
    .set_index('Referenced Tweet Author ID')
    .to_dict()['Afiliacion']
)

common_keys = list(set(mapa.keys()).intersection(set(mapa_2.keys())))
print(f"Hay {len(mapa)} usuarios identificados por la Silla")
print(f"Hay {len(mapa_2)} congresistas identificados según su partido")
print(f"Hay {len(common_keys)} usuarios identificados por ambas fuentes")
print('')
for key in common_keys:
    if mapa[key] != mapa_2[key]:
        # Usar el criterio de la silla vacía
        print(f"{user_name[key]}: Silla dice {mapa[key]} pero partido es {mapa_2[key]}")
    mapa_2.pop(key)

# Unir ambos diccioanrios
mapa_full = mapa_2 | mapa

# Eliminar los usuarios sin clasificar
mapa_full = {k:v for k,v in mapa_full.items() if v}
print('')
print(f"Total de usuarios clasificados según Silla + congresistas {len(mapa_full)}")

# Guardar
with open(os.path.join(path,"Pickle","User_Dicts","mapa_full.pkl"), "wb") as file:
    pickle.dump(mapa_full, file)

Hay 93 usuarios identificados por la Silla
Hay 230 congresistas identificados según su partido
Hay 15 usuarios identificados por ambas fuentes

JERobledo: Silla dice Centro pero partido es Izquierda
RoyBarreras: Silla dice Izquierda pero partido es Derecha
angelamrobledo: Silla dice Centro pero partido es Izquierda
AABenedetti: Silla dice Izquierda pero partido es Derecha
intiasprilla: Silla dice Izquierda pero partido es Centro

Total de usuarios clasificados según Silla + congresistas 299


# CHECKPOINT: Cargar PRESAMPLE

Load all pickle files will need

In [47]:
# Cargar Tweets PRE SAMPLE Tweets (elecciones 2019 y enero 2020)
retweets = pd.read_pickle(os.path.join(path,"Tweets_DataFrames",'retweets_pre.gzip'), compression='gzip')

# Cargar mapa de Silla + Congresistas
with open(os.path.join(path,"Pickle","User_Dicts","mapa_full.pkl"), "rb") as file:
    mapa = pickle.load(file)

In [48]:
# A cada retweet, le asignamos una afiliación según la persona retweteada
retweets["Party"] = retweets["Referenced Tweet Author ID"].map(mapa)
print(f"Cantidad de usuarios inicial en PRESAMPLE: {len(retweets['Author ID'].unique()):,.0f}")
# Hay mucho NA's values 
print(retweets["Party"].value_counts(dropna=False).apply(lambda x: f"{x:,}"))

# solo se tendrán en cuenta los tweets TESTIMONIO
#retweets = retweets[retweets["Party"].notna()]
print("")
print(f"Total de twitteros que retwitearon a alguien con clasificación {len(retweets['Author ID'].unique()):,}")
print("")
print(f"Total de Tweets TESTIMONIO (clasificados): {len(retweets):,.0f}")
retweets.head()

Cantidad de usuarios inicial en PRESAMPLE: 32,693
Party
NaN          5,671,099
Izquierda      578,238
Derecha        236,454
Centro         141,261
Name: count, dtype: object

Total de twitteros que retwitearon a alguien con clasificación 32,693

Total de Tweets TESTIMONIO (clasificados): 6,627,052


,Tweet ID,Author ID,Author Name,Referenced Tweet Author ID,Referenced Tweet Author Name,Referenced Tweet ID,Date,Party
0,1.349944e+18,7.099376e+17,RoseroLop,1.208053e+08,DanielPalam,1.349884e+18,2021/01/15 00:00:01,NaN
1,1.349944e+18,1.275312e+18,dsunnytears,1.958869e+08,RicciuP,1.349795e+18,2021/01/15 00:00:02,NaN
2,1.349944e+18,1.590730e+08,jatirado,1.034560e+18,jatirado_oc,1.349944e+18,2021/01/15 00:00:02,NaN
3,1.349944e+18,1.261210e+08,SoyAlexys,2.336361e+08,LiliEstupinanAc,1.349933e+18,2021/01/15 00:00:03,NaN
4,1.349944e+18,9.418440e+17,GirlLonelySoulx,9.738670e+17,ASTRO_Staff,1.349943e+18,2021/01/15 00:00:04,NaN


We create a 4x1 positive integer vector for every tweeter in the community that registers the number of RTs that the user has based on the political affilation. 

In [65]:
# We create lambda-functions that count the number of RTs for each political label.
a = lambda x: np.sum(x == "Derecha")
b = lambda x: np.sum(x == "Izquierda")
c = lambda x: np.sum(x == "Centro")

# given per political label for each user using the lambda-functions.
rts_usuario_paro = retweets.groupby("Author ID").agg({"Party": [a,b,c]})

rts_usuario_paro.columns = ["Retweets Derecha",
                            "Retweets Izquierda",
                            "Retweets Centro"]
rts_usuario_paro
# Total RTs...
rts_usuario_paro["Retweets Totales"] = rts_usuario_paro.sum(axis=1)

rts_usuario_paro["NPD"] = rts_usuario_paro['Retweets Totales'].apply(lambda x: 1 if x==0 else 0)

rts_usuario_paro.index = rts_usuario_paro.index.astype('float64')

# Now we determine the political affiliation by checking the index with the maximum.
rts_usuario_paro.sort_index()
print('Vector Database size is: ', rts_usuario_paro.shape)
rts_usuario_paro.head()

Vector Database size is:  (32693, 5)


,Retweets Derecha,Retweets Izquierda,Retweets Centro,Retweets Totales,NPD
Author ID,,,,,
12996.0,0,10,7,17,0
777978.0,0,0,0,0,1
784125.0,0,33,3,36,0
1061601.0,0,13,1,14,0
1488031.0,0,0,0,0,1


In [66]:
# Función de clasificación
def clasificar(row:pd.Series):
    # Revisar si segunda afiliación es igual a primera
    if row.nlargest(1).iloc[-1] == row.nlargest(2).iloc[-1]:
        return 'Sin Clasificar'
    else:
        return row.idxmax()

# Now we determine the political affiliation by checking the index with the maximum.
rts_usuario_paro["Afiliacion"] = rts_usuario_paro[["Retweets Centro", 
                                         "Retweets Derecha", 
                                         "Retweets Izquierda", 
                                         "NPD"]].apply(clasificar,axis=1)

conditions = [
    (rts_usuario_paro['Afiliacion'] == 'Retweets Izquierda'),
    (rts_usuario_paro['Afiliacion'] == 'Retweets Derecha'),
    (rts_usuario_paro['Afiliacion'] == 'Retweets Centro'),
    (rts_usuario_paro['Afiliacion'] == 'Sin Clasificar'),
    (rts_usuario_paro['Afiliacion'] == 'NPD')
]

choices = ['Izquierda', 'Derecha', 'Centro', 'Sin Clasificar', 'NPD']

rts_usuario_paro['Afiliacion'] = pd.Series(np.select(conditions, choices, default=''), index=rts_usuario_paro.index)

# We generate dummy variables for each political label...
rts_usuario_paro["Dummy Derecha"] = (rts_usuario_paro["Afiliacion"] == 'Derecha').astype('int32')
rts_usuario_paro["Dummy Izquierda"] = (rts_usuario_paro["Afiliacion"] == 'Izquierda').astype('int32')
rts_usuario_paro["Dummy Centro"] = (rts_usuario_paro["Afiliacion"] == 'Centro').astype('int32')
rts_usuario_paro["Dummy Sin Clasificar"] = (rts_usuario_paro["Afiliacion"] == 'Sin Clasificar').astype('int32')
rts_usuario_paro["Dummy NPD"] = (rts_usuario_paro["Afiliacion"] == 'NPD').astype('int32')

# We see the sizes of our groups
print("Afiliaciones de usuarios en el PRE SAMPLE")
print(rts_usuario_paro['Afiliacion'].value_counts(dropna = False).apply(lambda x: f"{x:,.0f}"))
rts_usuario_paro.head()

Afiliaciones de usuarios en el PRE SAMPLE
Afiliacion
Izquierda         13,782
NPD                7,960
Derecha            5,333
Centro             4,331
Sin Clasificar     1,287
Name: count, dtype: object


,Retweets Derecha,Retweets Izquierda,Retweets Centro,Retweets Totales,NPD,Afiliacion,Dummy Derecha,Dummy Izquierda,Dummy Centro,Dummy Sin Clasificar,Dummy NPD
Author ID,,,,,,,,,,,
12996.0,0,10,7,17,0,Izquierda,0,1,0,0,0
777978.0,0,0,0,0,1,NPD,0,0,0,0,1
784125.0,0,33,3,36,0,Izquierda,0,1,0,0,0
1061601.0,0,13,1,14,0,Izquierda,0,1,0,0,0
1488031.0,0,0,0,0,1,NPD,0,0,0,0,1


In [67]:
# Finally, we create a dictionary which stores the affiliation for each user.
user_to_party_paro = rts_usuario_paro.to_dict()['Afiliacion']

with open(os.path.join(path,"Pickle","User_Dicts","user_to_party_paro.pkl"), 'wb') as file:
    pickle.dump(user_to_party_paro,file)

rts_usuario_paro.to_pickle(os.path.join(path,"Pickle","User_Rts_Vector","rts_usuario_paro.pkl"))

# CHECKPOINT: Master Construction

In [76]:
# Información de usuarios del paro
users_information = pd.read_pickle(os.path.join(path, "Tweets_DataFrames","users_information.gzip"), compression='gzip').reset_index()

# vectores de clasificación para usuario PRESAMPLE
rts_usuario_paro = pd.read_pickle(os.path.join(path,"Pickle","User_Rts_Vector","rts_usuario_paro.pkl"))

# Resetar para hacer el merge
rts_usuario_paro = rts_usuario_paro.reset_index()
master = pd.merge(users_information,rts_usuario_paro,on='Author ID', how = 'outer',indicator=True)
master.loc[master['Afiliacion'].isna(),'Afiliacion'] = 'NPD'

# Preparar el master
master = (
    master.loc[:,('Author ID', 'Author Name', 'Afiliacion')]
    .rename(columns = {
        'Author ID':'User ID',
        'Author Name': 'Label',
        'Afiliacion': 'Political Affiliation'
    })
    .dropna()
    .drop_duplicates('User ID')
)
print(master['Political Affiliation'].value_counts().apply(lambda x: f"{x:,.0f}"))
print("")
print(f"Usuarios en el Master {len(master):,.0f}")
print("- \"NPD\" (No Prior Info): no se tienen tweets Testimonio del PRESAMPLE para clasificarlo")
print("- \"Sin Clasificar\": es clasificable pero tiene un empate en el top 2 del ranking de ideología ")
master.to_csv(os.path.join(path,'Master.csv'),index=False)

Political Affiliation
Izquierda         13,782
NPD               12,607
Derecha            5,331
Centro             4,330
Sin Clasificar     1,287
Name: count, dtype: object

Usuarios en el Master 37,337
- "NPD" (No Prior Info): no se tienen tweets Testimonio del PRESAMPLE para clasificarlo
- "Sin Clasificar": es clasificable pero tiene un empate en el top 2 del ranking de ideología 
